# ILIAD Pythonic Data Harmonisation Pipeline

### Imports

In [1]:
import pandas as pd
from rdflib import Graph, Namespace
from utils import bind_namespaces

from ontology_library.high_level_functions.high_level_observation_functions import (
    harmonise_oim_sosa_observation_sea_temperature_degree_celsius,
)

### Load sea temperature data into a DataFrame

In [2]:
# Load the JSON data into a DataFrame
df = pd.read_json("data/sea_temp.json")
df

,id,sea_temperature_celsius,timestamp,latitude,longitude
0,1234,4.6,2025-06-27 01:00:00+00:00,70.41,0.0
1,1235,5.1,2025-06-27 02:00:00+00:00,70.42,0.1


### Define a graph and bind namespaces for better readability

In [3]:
g = Graph()
g = bind_namespaces(g)

### Call the high-level harmonisation function for sea temperature for each row in the DataFrame

In [4]:
# Iterate over the rows and extract each field into variables
for index, row in df.iterrows():
    observation_id = str(row["id"])
    sea_temperature = row["sea_temperature_celsius"]
    timestamp = row["timestamp"]
    lat = row["latitude"]
    lon = row["longitude"]

    # Call the high-level function for harmonising sea temperature in OIM
    g += harmonise_oim_sosa_observation_sea_temperature_degree_celsius(
        sea_temperature=sea_temperature,
        observation_id=observation_id,
        result_time=timestamp,
        feature_of_interest_id=f"loc_{lat}_{lon}",
    )

### Print RDF output

In [5]:
print(g.serialize(format="turtle"))

@prefix oim-obs: <https://w3id.org/iliad/oim/observations/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudt_unit: <http://qudt.org/vocab/unit/> .
@prefix sosa: <http://www.w3.org/ns/sosa/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

oim-obs:1234 a sosa:Observation ;
    sosa:hasFeatureOfInterest <https://w3id.org/iliad/oim/features_of_interest/loc_70.41_0.0> ;
    sosa:hasResult <https://w3id.org/iliad/oim/results/2025-12-05_11:17:52.754307> ;
    sosa:observedProperty <https://w3id.org/iliad/oim/property/seaTemperature> ;
    sosa:resultTime "2025-06-27T01:00:00+00:00"^^xsd:dateTime .

oim-obs:1235 a sosa:Observation ;
    sosa:hasFeatureOfInterest <https://w3id.org/iliad/oim/features_of_interest/loc_70.42_0.1> ;
    sosa:hasResult <https://w3id.org/iliad/oim/results/2025-12-05_11:17:52.755628> ;
    sosa:observedProperty <https://w3id.org/iliad/oim/property/seaTemperature> ;
    sosa:resultTime "2025-06-27T02:00:00+00:00"^^xsd:dateTime .

<https://w3id.org/ili

### Write results to file

In [6]:
# Serialize the graph to a file
g.serialize(destination="output/sea_temp_observations_notebook.ttl", format="turtle")

print("✅ RDF data for sea temperature generated successfully.")

✅ RDF data for sea temperature generated successfully.
